In [20]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np

In [21]:
sc_adata = anndata.read_h5ad('./dataset1/scRNA.h5ad')
sc_adata

AnnData object with n_obs × n_vars = 10000 × 33691
    obs: 'celltype_final'
    var: 'features', 'n_cells'
    uns: 'celltype_mapper'
    layers: 'norm'

In [4]:
st_adata = anndata.read_h5ad('./dataset1/Spatial.h5ad')
st_adata

AnnData object with n_obs × n_vars = 1000 × 46732
    obs: 'cell_counts'
    uns: 'density'

In [30]:
df_obs = sc_adata.obs.copy()
selected_indices = []

for celltype, group in df_obs.groupby('celltype_final'):
    n = min(3, len(group))
    selected = group.sample(n=n, random_state=42)
    selected_indices.extend(selected.index.tolist())


sc_adata_sub = sc_adata[selected_indices].copy()

sc_exp = pd.DataFrame(
    sc_adata_sub.layers["norm"].toarray(),
    index=sc_adata_sub.obs_names,
    columns=sc_adata_sub.var_names
).T

In [31]:
sc_adata_sub.write_h5ad('./dataset1/scRNA_sub_sub3.h5ad')

In [ ]:
sc_exp.index.name = 'cell_id'
sc_exp.to_csv('raw_somatosensory_sc_exp_sub3cells.txt', sep='\t')
sc_exp

,TCAGCAATCAAGAATG-22L8TX_180927_001_B01,CGTCAAAAGAGAGTTT-32L8TX_181108_001_A01,GAGACCCTCTACGCGG-33L8TX_181108_001_B01,GGAGAACGTGCTCCGA-33L8TX_181108_001_B01,GTTCCGTAGTCAACAA-34L8TX_181108_001_C01,ACTTCCGTCCTCACTG-33L8TX_181108_001_B01,CTCCCTCCACAGTCAT-25L8TX_180927_003_A01,CATACCCGTTACACAC-27L8TX_180927_003_B01,AGACACTCAATGTGGG-28L8TX_180927_004_A01,AGGGTCCAGGACCCAA-33L8TX_181108_001_B01,...,GTTACAGGTGCAGGAT-35L8TX_181108_001_D01,TACCCGTAGCACCGAA-35L8TX_181108_001_D01,GTTGTAGTCACCCTCA-32L8TX_181108_001_A01,TCAGTGAGTGGGCTCT-35L8TX_181108_001_D01,ACCAAACGTTAGAAAC-34L8TX_181108_001_C01,ATCCACCCACTCTAGA-32L8TX_181108_001_A01,TCTGCCATCATGCCGG-24L8TX_180927_001_D01,TAGTGCAAGTGGAAAG-29L8TX_180927_004_B01,CGAGAAGTCTGTACAG-33L8TX_181108_001_B01,AGGCATTCATGACTCA-35L8TX_181108_001_D01
cell_id,,,,,,,,,,,,,,,,,,,,,
WASH7P,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
LOC105379213,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
LOC729737,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
LOC100996442,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
LOC105379214,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ND5,4.990333,0.0,0.0,0.0,0.000000,6.663736,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,6.472984
ND6,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
CYTB,7.692013,0.0,0.0,0.0,6.435613,7.579261,0.0,5.052727,0.0,0.0,...,5.880932,0.0,6.741095,6.138512,6.226384,4.378773,6.189775,0.0,6.673158,6.877934


In [ ]:
sc_anno = sc_adata_sub.obs[['celltype_final']]
sc_anno.to_csv('somatosensory_sc_labels_sub_sub3.txt', header=False, index=False)
sc_anno

,celltype_final
TCAGCAATCAAGAATG-22L8TX_180927_001_B01,Astro L1 FGFR3 SERPINI2
CGTCAAAAGAGAGTTT-32L8TX_181108_001_A01,Astro L1 FGFR3 SERPINI2
GAGACCCTCTACGCGG-33L8TX_181108_001_B01,Astro L1 FGFR3 SERPINI2
GGAGAACGTGCTCCGA-33L8TX_181108_001_B01,Astro L1-6 FGFR3 AQP1
GTTCCGTAGTCAACAA-34L8TX_181108_001_C01,Astro L1-6 FGFR3 AQP1
...,...
ATCCACCCACTCTAGA-32L8TX_181108_001_A01,Oligo L3-6 OPALIN LRP4-AS1
TCTGCCATCATGCCGG-24L8TX_180927_001_D01,Oligo L5-6 OPALIN LDLRAP1
TAGTGCAAGTGGAAAG-29L8TX_180927_004_B01,VLMC L1-5 PDGFRA COLEC12
CGAGAAGTCTGTACAG-33L8TX_181108_001_B01,VLMC L1-5 PDGFRA COLEC12


In [ ]:
st_exp = pd.DataFrame(
    st_adata.X,
    index=st_adata.obs_names,
    columns=st_adata.var_names,
)

st_exp = st_exp.T
st_exp.to_csv('Out_gene_expressions_10000genes.csv')

In [ ]:
num_spots = st_adata.n_obs
grid_size = int(np.ceil(np.sqrt(num_spots)))
step = 500
offset = 250

x_coords = []
y_coords = []
rect_codes = []
rect_indices = []

i = 0
for row in range(grid_size):
    for col in range(grid_size):
        if i >= num_spots:
            break
        x_coords.append(offset + step * row)
        y_coords.append(offset + step * col)
        rect_codes.append(f"{row}_{col}")
        rect_indices.append(i)
        i += 1

coords = pd.DataFrame({
    "Rect_code": rect_codes,
    "Rect_index": rect_indices,
    "X": x_coords,
    "Y": y_coords
}, index=st_adata.obs_names[:len(x_coords)])

coords[['X', 'Y']].to_csv('Out_rect_locations.csv')
